# import libs

In [1]:
import torch
import torch.nn
import torch.optim as optim
import random

In [2]:
torch.__version__

'1.1.0'

# device & seed setting

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# hyper parameter setting

In [5]:
# parameters
learning_rate = 0.001
training_epochs = 1
batch_size = 100

# Dataset setting

In [6]:
from mnist import *

In [7]:
mymnist = MyMNIST(batch_size)

# CNN Model setting

In [8]:
# CNN Model
class CNN(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        '''
        layer1 size:  torch.Size([-1, 32, 14, 14])
        layer2 size:  torch.Size([-1, 64, 7, 7])
        layer3 size:  torch.Size([-1, 128, 4, 4])
        view size:  torch.Size([-1, 2048])
        fc1 size:  torch.Size([-1, 625])
        fc2 size:  torch.Size([-1, 10])
        '''
        out = self.layer1(x)
        # print('layer1 size: ', out.size())
        out = self.layer2(out)
        # print('layer2 size: ', out.size())
        out = self.layer3(out)
        # print('layer3 size: ', out.size())
        out = out.view(out.size(0), -1)   # Flatten them for FC
        # print('view size: ', out.size())
        out = self.layer4(out)
        # print('fc1 size: ', out.size())
        out = self.fc2(out)
        # print('fc2 size: ', out.size())
        return out

In [9]:
model = CNN().to(device)

In [10]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2048, out_features=625, bias=True)
  (layer4): Sequential(
    (0): Linear(in_features=2048, out_features=625, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5)
  )
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)

In [11]:
# model(torch.randn(1, 1, 28, 28)) # for checking size

# define cost/loss & optimizer

In [12]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train my model

In [13]:
%%time
# train my model
total_batch = len(mymnist.train_data_loader)
model.train()    # set the model to train mode (dropout=True)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in mymnist.train_data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.189675093
Learning Finished!
Wall time: 12.5 s


# Test model and check accuracy

In [14]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    X_test = mymnist.mnist_test.data.view(len(mymnist.mnist_test), 1, 28, 28).float().to(device)
    Y_test = mymnist.mnist_test.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9759999513626099
